In [1]:
import requests
import bs4
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import pandas as pd

In [2]:
url_inicial = 'https://myb.ojs.inecol.mx/index.php/myb/issue/archive'
url_root = 'https://myb.ojs.inecol.mx/index.php/myb/issue/archive'
r=requests.get(url_inicial)

In [3]:
soup = BeautifulSoup(r.text, 'html.parser')

In [4]:
box = soup.find('div', class_='flex_container issues_list')
volumen=box.findAll('div',class_='obj_issue_summary')
vol = [x.find('a').get('href')for x in volumen]
vol

['https://myb.ojs.inecol.mx/index.php/myb/issue/view/246',
 'https://myb.ojs.inecol.mx/index.php/myb/issue/view/245',
 'https://myb.ojs.inecol.mx/index.php/myb/issue/view/243',
 'https://myb.ojs.inecol.mx/index.php/myb/issue/view/244',
 'https://myb.ojs.inecol.mx/index.php/myb/issue/view/242',
 'https://myb.ojs.inecol.mx/index.php/myb/issue/view/241',
 'https://myb.ojs.inecol.mx/index.php/myb/issue/view/240',
 'https://myb.ojs.inecol.mx/index.php/myb/issue/view/236',
 'https://myb.ojs.inecol.mx/index.php/myb/issue/view/235',
 'https://myb.ojs.inecol.mx/index.php/myb/issue/view/234',
 'https://myb.ojs.inecol.mx/index.php/myb/issue/view/233',
 'https://myb.ojs.inecol.mx/index.php/myb/issue/view/231',
 'https://myb.ojs.inecol.mx/index.php/myb/issue/view/232',
 'https://myb.ojs.inecol.mx/index.php/myb/issue/view/230',
 'https://myb.ojs.inecol.mx/index.php/myb/issue/view/229',
 'https://myb.ojs.inecol.mx/index.php/myb/issue/view/227',
 'https://myb.ojs.inecol.mx/index.php/myb/issue/view/228

In [5]:
vol2 =[]
for i in vol: 
    url_inicial1=i 
    r1 = requests.get(url_inicial1)
    soup1 = BeautifulSoup(r1.text, 'html.parser')
    box1 = soup1.find('section', class_='sections') 
    volumen1=box1.findAll('article', class_='article_summary')
    vol1 = [x.find('a').get('href')for x in volumen1]
    vol2+=vol1

In [6]:
vol2

['https://myb.ojs.inecol.mx/index.php/myb/article/view/2294',
 'https://myb.ojs.inecol.mx/index.php/myb/article/view/1438',
 'https://myb.ojs.inecol.mx/index.php/myb/article/view/2409',
 'https://myb.ojs.inecol.mx/index.php/myb/article/view/2089',
 'https://myb.ojs.inecol.mx/index.php/myb/article/view/2404',
 'https://myb.ojs.inecol.mx/index.php/myb/article/view/2268',
 'https://myb.ojs.inecol.mx/index.php/myb/article/view/1833',
 'https://myb.ojs.inecol.mx/index.php/myb/article/view/2381',
 'https://myb.ojs.inecol.mx/index.php/myb/article/view/2252',
 'https://myb.ojs.inecol.mx/index.php/myb/article/view/2060',
 'https://myb.ojs.inecol.mx/index.php/myb/article/view/2269',
 'https://myb.ojs.inecol.mx/index.php/myb/article/view/2318',
 'https://myb.ojs.inecol.mx/index.php/myb/article/view/1895',
 'https://myb.ojs.inecol.mx/index.php/myb/article/view/2279',
 'https://myb.ojs.inecol.mx/index.php/myb/article/view/2228',
 'https://myb.ojs.inecol.mx/index.php/myb/article/view/2242',
 'https:

In [7]:
def get_url_items(sopa,url):
    box1 = soup1.find('section', class_='sections') 
    volumen1=box1.findAll('article', class_='article_summary')
    vol1 = [x.find('a').get('href')for x in volumen1]
    return vol2

In [8]:
links_items=[]
i=0
while i<1:
    i+=1
    print(f'Estoy en la pagina {url_inicial}')
    r_pag=requests.get(url_inicial)
    s_p=BeautifulSoup(r_pag.text,'html.parser')
    links=get_url_items(s_p, url_inicial)
    links_items.append(links) 
    next_a=s_p.select('li.next > a')
    if not next_a or not next_a[0].get('href'):
        break

Estoy en la pagina https://myb.ojs.inecol.mx/index.php/myb/issue/archive


In [9]:
list_scraper=[]
for i in links_items:
    for j in i:
        list_scraper.append(j)
len(list_scraper)

396

In [10]:
uno=list_scraper[0]
r_item=requests.get(uno)
s_item=BeautifulSoup(r_item.text, 'html.parser')

In [11]:
#funcion para iniciar el escraper de cada libro
def scraper_book(url):
    content_book={}
    r=requests.get(url)
    tire='Madera y Bosques'
    a='VI. Biotecnología y Ciencias Agropecuarias'
    tem='Recursos Forestales'
    s_item=BeautifulSoup(r.text,'html.parser')
    #titulo de revista
    try:
        titulo=tire
        content_book['Titulo Revista']=tire
    except AttributeError:
        content_book['Titulo Revista']=None
    #area
    try:
        area=a
        content_book['Area']=a
    except AttributeError:
        content_book['Area']=None
    #tematica
    try:
        tema=tem
        content_book['Tematica']=tem
    except AttributeError:
        content_book['Tematica']=None
    #titulo articulo
    try:
        titu=s_item.find('h1', class_='page_title article-full-title').get_text(strip=True)
        content_book['Titulo Articulo']=titu
    except AttributeError:
        content_book['Titulo Articulo']=None
    #resumen
    try:
        resu=s_item.find('div', class_='abstract').get_text(strip=True)
        content_book['Resumen']=resu.replace("Resumen","")
    except AttributeError:
        content_book['Resumen']=None
    #abstract
    try:
        abst=s_item.find('div', class_='item cover_image').get_text(strip=True)
        content_book['Abstract']=abst
    except AttributeError:
        content_book['Abstract']=None
    #Link incial
    try:
        linkart=url
        content_book['Link Articulo']=linkart
    except AttributeError:
        content_book['Link Articulo']=None
    #link articulo
    try:
        link=s_item.find('a', class_='btn galley-link obj_galley_link pdf').get('href')
        content_book['Link PDF']=urljoin(url_root, link)
    except AttributeError:
        content_book['Link PDF']=None
    return content_book

In [12]:
list_scraper=list_scraper[0:396]
datos_book=[]
for idx, i in enumerate(list_scraper):
    print(f'estas escrapeando la pag {idx}')
    datos_book.append(scraper_book(i))

estas escrapeando la pag 0
estas escrapeando la pag 1
estas escrapeando la pag 2
estas escrapeando la pag 3
estas escrapeando la pag 4
estas escrapeando la pag 5
estas escrapeando la pag 6
estas escrapeando la pag 7
estas escrapeando la pag 8
estas escrapeando la pag 9
estas escrapeando la pag 10
estas escrapeando la pag 11
estas escrapeando la pag 12
estas escrapeando la pag 13
estas escrapeando la pag 14
estas escrapeando la pag 15
estas escrapeando la pag 16
estas escrapeando la pag 17
estas escrapeando la pag 18
estas escrapeando la pag 19
estas escrapeando la pag 20
estas escrapeando la pag 21
estas escrapeando la pag 22
estas escrapeando la pag 23
estas escrapeando la pag 24
estas escrapeando la pag 25
estas escrapeando la pag 26
estas escrapeando la pag 27
estas escrapeando la pag 28
estas escrapeando la pag 29
estas escrapeando la pag 30
estas escrapeando la pag 31
estas escrapeando la pag 32
estas escrapeando la pag 33
estas escrapeando la pag 34
estas escrapeando la pag 35
es

In [13]:
df_catalogo=pd.DataFrame(datos_book)
df_catalogo

,Titulo Revista,Area,Tematica,Titulo Articulo,Resumen,Abstract,Link Articulo,Link PDF
0,Madera y Bosques,VI. Biotecnología y Ciencias Agropecuarias,Recursos Forestales,Cambio de uso de suelo y fragmentación del pai...,La deforestación y fragmentación son considera...,None,https://myb.ojs.inecol.mx/index.php/myb/articl...,https://myb.ojs.inecol.mx/index.php/myb/articl...
1,Madera y Bosques,VI. Biotecnología y Ciencias Agropecuarias,Recursos Forestales,Crecimiento de Pinus patula var. longepeduncul...,Las investigaciones sobre crecimiento y rendim...,None,https://myb.ojs.inecol.mx/index.php/myb/articl...,https://myb.ojs.inecol.mx/index.php/myb/articl...
2,Madera y Bosques,VI. Biotecnología y Ciencias Agropecuarias,Recursos Forestales,¿Una fase de endurecimiento más prolongada mej...,"El algarrobo blanco (Prosopis alba), especie a...",None,https://myb.ojs.inecol.mx/index.php/myb/articl...,https://myb.ojs.inecol.mx/index.php/myb/articl...
3,Madera y Bosques,VI. Biotecnología y Ciencias Agropecuarias,Recursos Forestales,"Bonos de carbono, una alternativa de sostenibi...",Los pagos por servicios ambientales son parte ...,None,https://myb.ojs.inecol.mx/index.php/myb/articl...,https://myb.ojs.inecol.mx/index.php/myb/articl...
4,Madera y Bosques,VI. Biotecnología y Ciencias Agropecuarias,Recursos Forestales,Barreras para la producción y el mercado de bi...,La bioenergía es una alternativa importante pa...,None,https://myb.ojs.inecol.mx/index.php/myb/articl...,https://myb.ojs.inecol.mx/index.php/myb/articl...
...,...,...,...,...,...,...,...,...
391,Madera y Bosques,VI. Biotecnología y Ciencias Agropecuarias,Recursos Forestales,Cambios de cobertura/uso del suelo en una porc...,La Porción Surponiente del Estado de México (P...,None,https://myb.ojs.inecol.mx/index.php/myb/articl...,https://myb.ojs.inecol.mx/index.php/myb/articl...
392,Madera y Bosques,VI. Biotecnología y Ciencias Agropecuarias,Recursos Forestales,Biología de Lycoriella ingenua y Bradysia impa...,Se estudiaron dos especies de mosquito fungoso...,None,https://myb.ojs.inecol.mx/index.php/myb/articl...,https://myb.ojs.inecol.mx/index.php/myb/articl...
393,Madera y Bosques,VI. Biotecnología y Ciencias Agropecuarias,Recursos Forestales,Densidad básica de la madera de dos pinos y su...,Para identificar las diferencias en densidad b...,None,https://myb.ojs.inecol.mx/index.php/myb/articl...,https://myb.ojs.inecol.mx/index.php/myb/articl...
394,Madera y Bosques,VI. Biotecnología y Ciencias Agropecuarias,Recursos Forestales,Efecto del tratamiento higro-térmico en las ca...,La madera del géneroQuercuspresenta un potenci...,None,https://myb.ojs.inecol.mx/index.php/myb/articl...,https://myb.ojs.inecol.mx/index.php/myb/articl...


In [14]:
df_catalogo.to_csv('Revista027.01.csv', index=False)